In [1]:
import warnings
warnings.filterwarnings("ignore")

## Research various commodity spread ideas

In [280]:

import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import peakutils
from peakutils.plot import plot as pplot
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema


import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import ipdb
import traceback
import pandas_datareader.data as pdr
from dashapp import dashapp2 as dashapp
import dash_html_components as html
import dash_core_components as dcc
from itertools import accumulate 
import pathlib
import shutil
import urllib.request as request
from contextlib import closing
import zipfile

importlib.reload(dashapp)

2020-06-07 10:03:41,155 - matplotlib.pyplot - DEBUG - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


<module 'dashapp.dashapp2' from '/Users/bperlman1/Documents/billybyte/pyliverisk/dashapp/dashapp/dashapp2.py'>

In [3]:
from pandas.tseries.holiday import USFederalHolidayCalendar
bday_us = pd.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())


2020-06-03 14:05:09,873 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


### important global variables

In [4]:

DEBUG_IT=False
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'


#### get all contracts in the options database

In [5]:
pga = db_info.get_db_info()
print(f"futtab max date: {pga.get_sql(f'select max(settle_date) from {futtab}')}")
print(f"opttab max date: {pga.get_sql(f'select max(settle_date) from {opttab}')}")


  sec_db
futtab max date:         max
0  20200602
opttab max date:         max
0  20200602


In [6]:
def _create_batch_indices(l,n):
    ii = list(range(l))
    num = n
    # list of length in which we have to split 
    length_to_split = list(np.repeat(num,len(ii)/num )) + [len(ii)%num]
    print(length_to_split)

    # Using islice 
    r = [ii[x - y: x] for x, y in zip( 
              accumulate(length_to_split), length_to_split)] 
    return r
_create_batch_indices(11,3)

[3, 3, 3, 2]


[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10]]

In [340]:
def str_to_date(d,sep='-'):
    try:
        dt = datetime.datetime.strptime(str(d)[:10],f'%Y{sep}%m{sep}%d')
    except:
        return None
    return dt

def get_local_peaks_and_valleys(df_in,n,col='close'):
    '''
    :param df - Input DataFrame
    :param n - number of data points that surround peak and valley
    :param col - the data column to observe
    '''
    df = df_in.copy()
    # Find local peaks
    df['valley'] = df.iloc[argrelextrema(df[col].values, np.less_equal, order=n)[0]][col]
    df['peak'] = df.iloc[argrelextrema(df[col].values, np.greater_equal, order=n)[0]][col]
    return df
 
def get_draw_downs(df_in,close='close'):
    df = df_in.copy()
    df['peak'] = df[close].expanding(min_periods=1).max()
    df['drawdown'] = df.peak-df[close]
    return df.drawdown
    

In [7]:
def get_spread(contract_front,contract_back):
    sql = f"""
    with 
    f1 as (
        select *
        from {futtab} 
        where symbol in ('{contract_front}','{contract_back}')
    )
    select * from f1 
    """
    df = pga.get_sql(sql)
    df1 = df[df.symbol==contract_front].copy()
    df2 = df[df.symbol==contract_back].copy()
    df1['front'] = df1.close
    df2['back'] = df2.close
    df_both = df1[['settle_date','front']].merge(
        df2[['settle_date','back']],on='settle_date',how='inner')
    df_both['spread'] = df_both.front - df_both.back
    return df_both

MONTH_CODES = 'FGHJKMNQUVXZ'
DICT_MONTH_CODE = {MONTH_CODES[i]:i+1 for i in range(len(MONTH_CODES))}

def get_CL_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month -1
    if month<1:
        month = 12
        year = year - 1
    return datetime.datetime(year,month,25) - 4*bday_us

def get_NG_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return datetime.datetime(year,month,1) - 3*bday_us


In [85]:
get_CL_expiry('CLM20'),get_NG_expiry('NGM20')

(Timestamp('2020-05-19 00:00:00'), Timestamp('2020-05-27 00:00:00'))

In [7]:
def _commod_list():
    sql = f"""select distinct substring(symbol,1,2) commod from {futtab}
    order by substring(symbol,1,2) """
    df = pga.get_sql(sql)
    return df

In [8]:
df_commod = _commod_list()

In [394]:
def _commod_month_list():
    sql = f"""select distinct symbol from {futtab}"""
    df = pga.get_sql(sql)
    df['commod'] = df.symbol.str[0:2]
    df['year'] = df.symbol.str[-2:].astype(int)
    df['mc'] = df.symbol.str[-3]
    df = df[['commod','year','mc']].sort_values(['commod','year','mc']).drop_duplicates()
    df.index = list(range(len(df)))
    df['sym'] = df.commod+df.year.astype(str)+df.mc
    return df

df_commod_month = _commod_month_list()


## Example of showing dashapp.plotly_plot with reverse order descending x axis
The xaxis values must be numbers, and should be consecutive

In [11]:
# syms = ['SIK20','SIZ99']
syms = ['ZCN19','ZCZ19']
sym = ','.join([f"'{v}'" for v in syms])
df_crude = pga.get_sql(f"select symbol,settle_date, close from {futtab} where symbol in ({sym})")
df_crude['settle_dt'] = [dashapp.str_to_date(str(d),sep='') for d in df_crude.settle_date.values]
max_date = df_crude.settle_dt.max()
df_crude['dte'] = (max_date-df_crude.settle_dt).dt.days
df_plot = df_crude[df_crude.symbol==syms[0]].iloc[-100:]
df_plot[syms[0]] = df_plot.close
for s in syms[1:]:
    values = df_crude[df_crude.symbol==s].iloc[-100:].close.values
    if 'Z99' in s:
        values = values/10
    df_plot[s] = values
f1 = dashapp.plotly_plot(
    df_in=df_plot[['dte']+syms],x_column='dte',y_left_label='Closing Price',
    plot_title=f'{sym} Price vs Days To Expiration'
)
f1.update_layout(xaxis={'type':'-','autorange':'reversed','title':'Days To Expiry'})
f2 = dashapp.plotly_plot(
    df_in=df_plot[['settle_date']+syms],x_column='settle_date',y_left_label='Closing Price',
    plot_title=f'{sym} Price vs Settle Date'
)
iplot(f1)
iplot(f2)

In [32]:
for y in range(11,21):
    df_zcn20_zcz20 = get_spread(f'ZCN{y}',f'ZCZ{y}').iloc[-120:]
    iplot(dashapp.plotly_plot(df_in=df_zcn20_zcz20,x_column='settle_date',
                      yaxis2_cols=['spread'],y_right_label='spread',
                              y_left_label='price',
                             plot_title=f'ZCN{y} ZCZ{y}'))

In [34]:
import importlib
importlib.reload(dashapp)
df_c = pga.get_sql(f"select * from {futtab} where symbol='ZCN20'").iloc[-120:]
iplot(dashapp.PlotlyCandles(df_c,date_column='settle_date',title="ZCN20").get_figure())